# フレア検出 学習ノート 02: フレア検出とエネルギー計算

このノートでは、`BaseFlareDetector` が行っている
フレア検出 (`flaredetect`, `flaredetect_check`) とエネルギー計算
(`tess_band_energy`, `flare_energy`) の流れを詳しく学びます。


## このノートで学ぶこと（予定）

- デトレンド後フラックス `s2mPDCSAPflux` と誤差配列 `mPDCSAPfluxerr_cor` の役割
- しきい値 `5σ` / `1σ` に基づくフレア候補抽出ロジック
- `flaredetect_check` によるベースライン補正とフレア継続時間の推定
- TESS 応答関数を用いたエネルギー換算の考え方
- エネルギー閾値 `ene_thres_low` / `ene_thres_high` の意味とチューニング例


## 1. 概要と前提

このノートでは、すでに `process_data=True` で一通り処理された
`FlareDetector_EK_Dra` インスタンスを出発点にして、

- デトレンド後フラックス `s2mPDCSAPflux`
- 誤差配列 `mPDCSAPfluxerr_cor`
- フレア検出結果（`detecttime`, `starttime`, `endtime`, `peaktime`）
- フレアエネルギー配列 `energy` とその集約結果

がどのように計算されているかを、コードと図を通じて確認していきます。

前提として：

- `data/TESS/EK_Dra/` 以下に TESS LC FITS ファイルが配置されていること
- `src/base_flare_detector.py` / `src/flarepy_EK_Dra.py` がインポート可能な状態であること
- `flare_learning_01_data_preprocessing.ipynb` で、
  Path まわりや FITS 構造のイメージをつかんでいること

を想定しています。


In [ ]:
"""EK Dra の 1 ファイルを選び、process_data=True で一括処理したインスタンスを作る。"""

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from src.flarepy_EK_Dra import FlareDetector_EK_Dra

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
print("DATA_DIR:", DATA_DIR)
print("DATA_DIR.exists():", DATA_DIR.exists())

fits_files = sorted(DATA_DIR.glob("*.fits")) if DATA_DIR.exists() else []
print("found", len(fits_files), "FITS files")

example_file = fits_files[0] if fits_files else None
print("example_file:", example_file)

if example_file is None:
    detector = None
    print("FITS ファイルが見つからないため、以降のセルはスキップしてください。")
else:
    detector = FlareDetector_EK_Dra(
        file=str(example_file),
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )
    detector.show_variables()

### `process_data=True` が内部でしていること（ざっくり）

`FlareDetector_EK_Dra(..., process_data=True)` と書くと、
コンストラクタの中で `BaseFlareDetector.process_data()` が呼ばれ、
おおまかに次の処理が順番に行われます。

1. `remove()`
   - 恒星ごとのサブクラスでトランジットなどを除去するフック。
   - EK Dra では特別な処理をしておらず、そのまま通過します。
2. `apply_gap_correction()`
   - 観測のギャップ（大きな欠測間隔）を検出し、
     その前後でフラックスのオフセットが飛ばないようにつなぎ直します。
3. `detrend_flux()`
   - 長周期の変動をローパスフィルタ＋スプラインで推定し、
     それを引き算することでフレアの短時間変動を強調します。
4. `reestimate_errors()`
   - フレアのないと思われる静穏な区間から、
     時刻ごとのノイズレベル（誤差）を再推定します。
5. `flaredetect()` / `flaredetect_check()`
   - `5σ` 以上の連続した点をフレア候補として拾い上げ、
     前後の `1σ` 区間などを使ってフレア開始・終了・ピークを決め直します。
6. `calculate_precise_obs_time()`
   - 大きなギャップを除いた実効観測時間を計算します。
7. `flare_energy()`
   - 各フレアのエネルギー配列 `energy` から、
     閾値 `ene_thres_low`〜`ene_thres_high` でフィルタして
     フレア数と総エネルギーを計算します。
8. `flux_diff()` / `rotation_period()`
   - 星黒点の振幅や回転周期などを求めます。

このノートでは、このうち **3〜7 の部分**を中心に、
実際の `detector` インスタンスから配列を取り出して眺めていきます。


## 2. デトレンド後フラックスと誤差再推定

ここでは、`process_data()` の中でも特にフレア検出に直結する

- デトレンド後フラックス `s2mPDCSAPflux`
- 誤差配列 `mPDCSAPfluxerr_cor`

の中身を確認します。

イメージとしては次のような流れになっています。

1. 元の正規化光度 `mPDCSAPflux` には、
   - 回転などの比較的ゆっくりした変動
   - フレアのような急な変動
     が両方混ざっています。
2. `detrend_flux()` では、
   - ローパスフィルタとスプライン補間で「ゆっくりした変動（ベースライン）」を推定
   - それを引き算して、短時間変動だけを取り出したものが `s2mPDCSAPflux`
3. `reestimate_errors()` では、
   - フレアがなさそうな "静かな" 区間をスライディングウィンドウで探し
   - 各時刻ごとのノイズレベル（1σ）を推定したものが `mPDCSAPfluxerr_cor`

以下のセルでは、まず `detector` からこれらの配列を取り出して、
小さな表と簡単なプロットで確認します。


In [ ]:
if detector is None:
    print("detector がまだ作成されていません。上のセルを先に実行してください。")
else:
    print("len(tessBJD):", len(detector.tessBJD))
    print("len(mPDCSAPflux):", len(detector.mPDCSAPflux))
    print("len(s2mPDCSAPflux):", len(detector.s2mPDCSAPflux) if detector.s2mPDCSAPflux is not None else None)
    print("len(mPDCSAPfluxerr):", len(detector.mPDCSAPfluxerr) if detector.mPDCSAPfluxerr is not None else None)
    print("len(mPDCSAPfluxerr_cor):", len(detector.mPDCSAPfluxerr_cor) if detector.mPDCSAPfluxerr_cor is not None else None)

    if detector.s2mPDCSAPflux is not None and detector.mPDCSAPfluxerr_cor is not None:
        preview = pd.DataFrame(
            {
                "tessBJD": detector.tessBJD[:8],
                "mPDCSAPflux": detector.mPDCSAPflux[:8],
                "s2mPDCSAPflux": detector.s2mPDCSAPflux[:8],
                "err_orig": detector.mPDCSAPfluxerr[:8],
                "err_reest": detector.mPDCSAPfluxerr_cor[:8],
            }
        )
        preview

In [ ]:
if detector is None or detector.s2mPDCSAPflux is None:
    print("detector がまだ作成されていないか、s2mPDCSAPflux が未計算です。")
else:
    # 元の正規化光度
    fig_raw = px.line(
        x=detector.tessBJD,
        y=detector.mPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Normalized Flux"},
        title="02-1: Raw normalized flux (mPDCSAPflux)",
    )
    fig_raw.show()

    # デトレンド後フラックス
    fig_det = px.line(
        x=detector.tessBJD,
        y=detector.s2mPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Detrended Flux"},
        title="02-2: Detrended flux (s2mPDCSAPflux)",
    )
    fig_det.add_hline(y=0.0, line_color="gray", line_dash="dash")
    fig_det.show()

### 図の読み方

- 1 つ目の図（Raw normalized flux）
  - 回転によるゆっくりした変動と、フレアと思われる鋭いピークが両方見えます。
- 2 つ目の図（Detrended flux）
  - ゆっくりした変動がほぼ取り除かれ、ベースラインが 0 付近にそろっています。
  - フレアのピークだけが目立つ形になるため、しきい値で検出しやすくなります。

`reestimate_errors()` で求められた `mPDCSAPfluxerr_cor` は、
このデトレンド後フラックスに対して「ここから何 σ 上に出ていればフレア候補か」を
決めるための基準になります。


## 3. `flaredetect` のアルゴリズム

`flaredetect()` は、デトレンド後フラックス `s2mPDCSAPflux` と
誤差配列 `mPDCSAPfluxerr_cor` を使って、

- 「5σ 以上」の連続した点の塊をフレア候補として見つける
- その前後を 1σ レベルで広げて、フレアの開始・終了時刻を見積もる

といった処理を行っています。

ここでは、実際に `detector` が検出したフレア一覧を
表と図で眺めてみます。


In [ ]:
if detector is None or detector.energy is None:
    print("detector が未定義か、エネルギー配列がまだありません。")
else:
    n = min(
        len(detector.detecttime),
        len(detector.starttime),
        len(detector.endtime),
        len(detector.peaktime),
        len(detector.energy),
    )
    print("検出フレア数 (配列長の最小値):", n)

    if n == 0:
        print("このファイルではフレアが検出されていません。")
    else:
        df_flares = pd.DataFrame(
            {
                "detecttime": detector.detecttime[:n],
                "starttime": detector.starttime[:n],
                "endtime": detector.endtime[:n],
                "peaktime": detector.peaktime[:n],
                "energy": detector.energy[:n],
            }
        )
        df_flares.head()

In [ ]:
if detector is None or detector.energy is None or len(detector.energy) == 0:
    print("フレアが検出されていないため、このセルはスキップします。")
else:
    # デトレンド後フラックスに、検出されたフレアの開始・終了・ピークを重ねて表示
    fig = px.line(
        x=detector.tessBJD,
        y=detector.s2mPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Detrended Flux"},
        title="03: Detrended flux with detected flares",
    )

    # 開始・終了時刻の間を薄い帯で示す
    for st, ed in zip(detector.starttime, detector.endtime):
        fig.add_vrect(
            x0=st,
            x1=ed,
            fillcolor="rgba(255, 165, 0, 0.2)",
            line_width=0,
        )

    # ピーク位置を点で示す
    fig.add_scatter(
        x=detector.peaktime,
        y=[0] * len(detector.peaktime),
        mode="markers",
        marker=dict(color="red", size=8),
        name="flare peaks (on baseline)",
    )

    fig.show()

## 4. `flaredetect_check` とフレア継続時間

`flaredetect()` が見つけたフレア候補に対して、
`flaredetect_check()` は次のような追加処理を行います。

- フレアの前後に直線ベースラインをフィットし、
  そのベースラインからの差分としてフレア形状をとらえ直す。
- ベースラインから 1σ 以上上に出ている区間を正確に抽出し、
  フレアの継続時間 `duration` を計算する。

ノートブックから見える形では、

- `detector.duration`
- `detector.edecay`（減衰時間スケール）

といった配列として結果が保存されています。


In [ ]:
if detector is None or detector.duration is None:
    print("detector が未定義か、duration がまだ計算されていません。")
else:
    print("len(duration):", len(detector.duration))
    if len(detector.duration) == 0:
        print("フレアが検出されていないか、duration が空です。")
    else:
        df_dur = pd.DataFrame(
            {
                "starttime": detector.starttime,
                "endtime": detector.endtime,
                "duration": detector.duration,
                "edecay": detector.edecay,
                "energy": detector.energy,
            }
        )
        df_dur.head()

In [ ]:
if detector is None or detector.duration is None or len(detector.duration) == 0:
    print("フレアが検出されていないため、このセルはスキップします。")
else:
    fig_hist_dur = px.histogram(
        x=detector.duration,
        nbins=30,
        labels={"x": "Duration [day]", "y": "Count"},
        title="04: Distribution of flare durations",
    )
    fig_hist_dur.show()

    fig_scatter_dur = px.scatter(
        x=detector.duration,
        y=detector.energy,
        labels={"x": "Duration [day]", "y": "Energy [erg]"},
        title="04: Flare energy vs duration",
    )
    fig_scatter_dur.update_yaxes(type="log")
    fig_scatter_dur.show()

## 5. エネルギー計算 (`tess_band_energy`, `flare_energy`)

`BaseFlareDetector` では、検出された各フレアについて

- まずフレアの「フラックス増加量（count）」を積分
- それを TESS 応答関数と恒星パラメータ（半径・温度）を用いて
  エネルギー [erg] に変換

することで、`energy` 配列を作っています。

- `tess_band_energy(count)`
  - TESS の波長応答関数と、星の有効温度・半径から
    「ある count がどれくらいのエネルギーに相当するか」を計算する関数です。
- `flare_energy(ene_thres_low, ene_thres_high)`
  - `energy` 配列に対して、指定したエネルギー範囲でフィルタリングを行い、
    フレア数と総エネルギーを集計します。


In [ ]:
if detector is None or detector.energy is None:
    print("detector が未定義か、energy がまだ計算されていません。")
else:
    print("len(energy):", len(detector.energy))
    print("ene_thres_low:", detector.ene_thres_low)
    print("ene_thres_high:", detector.ene_thres_high)
    print("flare_number:", detector.flare_number)
    print("sum_flare_energy:", detector.sum_flare_energy)

    if len(detector.energy) > 0:
        fig_energy_hist = px.histogram(
            x=detector.energy,
            nbins=40,
            labels={"x": "Flare Energy [erg]", "y": "Count"},
            title="05: Flare energy distribution (single file)",
        )
        fig_energy_hist.update_xaxes(type="log")
        fig_energy_hist.show()

## 6. エネルギー閾値のチューニング例

最後に、エネルギー閾値 `ene_thres_low` / `ene_thres_high` を変えると

- フレア数 `flare_number`
- 総フレアエネルギー `sum_flare_energy`

がどのように変わるかを簡単に見てみます。

ここでは、同じ `energy` 配列に対して

- 下限 `ene_thres_low` だけを複数パターンで変化させる
- 上限 `ene_thres_high` は一定のままにする

という実験を行います。


In [ ]:
if detector is None or detector.energy is None or len(detector.energy) == 0:
    print("フレアが検出されていないため、このセルはスキップします。")
else:
    lows = np.logspace(33, 35, 9)  # 1e33 〜 1e35 を対数スケールでサンプリング
    high = detector.ene_thres_high

    rows = []
    for low in lows:
        # flare_energy はインスタンスの状態を書き換えるので、
        # 毎回同じ energy 配列に対して呼び出して結果だけ記録します。
        detector.flare_energy(energy_threshold_low=low, energy_threshold_high=high)
        rows.append(
            {
                "ene_thres_low": low,
                "ene_thres_high": high,
                "flare_number": detector.flare_number,
                "sum_flare_energy": detector.sum_flare_energy,
            }
        )

    df_thres = pd.DataFrame(rows)
    df_thres

In [ ]:
if "df_thres" not in globals():
    print("まず 6. の前のセルを実行して df_thres を作成してください。")
else:
    fig_thres = px.line(
        df_thres,
        x="ene_thres_low",
        y="flare_number",
        markers=True,
        labels={"ene_thres_low": "Energy threshold low [erg]", "flare_number": "Number of flares"},
        title="06: Number of flares vs energy threshold (low)",
    )
    fig_thres.update_xaxes(type="log")
    fig_thres.show()

    fig_sum = px.line(
        df_thres,
        x="ene_thres_low",
        y="sum_flare_energy",
        markers=True,
        labels={"ene_thres_low": "Energy threshold low [erg]", "sum_flare_energy": "Sum flare energy [erg]"},
        title="06: Sum flare energy vs energy threshold (low)",
    )
    fig_sum.update_xaxes(type="log")
    fig_sum.update_yaxes(type="log")
    fig_sum.show()